In [1]:
import json, urllib
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
%matplotlib inline
import scipy.stats as ss
import numpy as np

In [2]:
df= pd.read_csv('data_final_project_v3.csv', sep=';', encoding='ISO-8859-1')
df.head()

,directorio,llave,fex_c,ciudad_origen,depto_origen,mes_recolec,trimestre,sexo,edad,rango_edad,parentesco,estado_civ,estrato,ingreso_hogar,rango_ingreso_hog,estudia,nivel_edu,diploma,labor,viajo,extranjero,cuantos_ext,interno,cuantos_int,depto_dest,mpio_dest,fecha_ini,fecha_fin,pernoct,motivo_int,depto_mv,mpio_mv,tiene_viv,aloj_segviv,aloj_hotel,aloj_centvac,aloj_fami,aloj_vivtur,med_transp,pago_paqtur,quien_pago,pers_gasto,cuanto_pago,estim_valor,paqtur_aloj,paqtur_transp,paqtur_alim,paqtur_tours,paqtur_segur,paqtur_otros,paqtur_recrea,gasto_tot,motivo_no,gast_aloj,valgast_aloj,pergast_aloj,gast_transp,valgast_transp,pergast_transp,gast_public,valgast_public,pergast_public,gast_alim,valgast_alim,pergast_alim,gast_aereo,valgast_aereo,pergast_aereo,excursion,excur_extranj,excur_int,excur_motiv,excur_deptodest,excur_mpiodest,excur_medtransp,excur_valgasttransp,excur_pergasttransp,excur_valgastpublic,excur_pergastpublic,excur_valgastalim,excur_pergastalim,Unnamed: 81
0,3000000,3000000_1,2629.5,Medellín,Antioquia,FEBRERO,PRIMERO,mujer,56,55_60,jefe,soltero,1,880000,1_2,no,primaria,NaN,ocupado,no,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3000000,3000000_1,2629.5,Medellín,Antioquia,FEBRERO,PRIMERO,mujer,25,20_25,hijo,soltero,1,880000,1_2,si,media,bachiller,desocupado,no,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recursos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3000000,3000000_1,2629.5,Medellín,Antioquia,FEBRERO,PRIMERO,mujer,1,0_15,nieto,NaN,1,880000,1_2,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3000001,3000001_1,2256.5,Medellín,Antioquia,FEBRERO,PRIMERO,mujer,39,35_40,jefe,separado,1,860000,1_2,no,secundaria,NaN,ocupado,no,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3000001,3000001_1,2256.5,Medellín,Antioquia,FEBRERO,PRIMERO,mujer,14,0_15,hijo,soltero,1,860000,1_2,si,secundaria,NaN,inactivo,no,NaN,0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,recursos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:

def tipo(x):
    if x['edad'].count() < 2:
        return x['sexo'][0]
    else:
        return "familia"

def rango_edad(x):
    mediana=np.floor(x['edad'].median()) 
    if mediana > 80:
        return "80_"
    elif mediana >= 75:
        return '75_80'
    elif mediana >= 70:
        return '70_75'
    elif mediana >= 65:
        return '65_70'
    elif mediana >= 60:
        return '60_65'
    elif mediana >= 55:
        return '55_60'
    elif mediana >= 50:
        return '50_55'
    elif mediana >= 45:
        return '45_50'
    elif mediana >= 40:
        return '40_45'
    elif mediana >= 35:
        return '35_40'
    elif mediana >= 30:
        return '30_35'
    elif mediana >= 25:
        return '25_30'
    elif mediana >= 20:
        return '20_25'
    elif mediana >= 15:
        return '15_20'
    elif mediana >= 10:
        return '10_15'
    else:
        return '0_10'
        
def estado_civil(x):
    if x['edad'].count() < 2:
        return x['estado_civ'][0]
    else:
        return "familia"

def alojamiento(x):
    if 'si' in x['aloj_hotel']:
        return 'hotel'
    elif 'si' in x['aloj_centvac']:
        return 'centro_vac'
    elif 'si' in x['aloj_vivtur']:
        return 'vivienda_tur'
    elif 'si' in x['aloj_segviv']:
        return 'segunda_viv'
    elif 'si' in x['aloj_fami']:
        return 'casa_familia'
    else:
        return None

def educacion(x):
    if 'superior' in x['nivel_edu'].tolist() and 'posgrado' in x['diploma'].tolist():
        return "superior_posgrado"
    elif 'superior' in x['nivel_edu'].tolist() and 'universitario' in x['diploma'].tolist():
        return "superior_univerisitario"
    elif 'superior' in x['nivel_edu'].tolist() and 'tecnico' in x['diploma'].tolist():
        return "superior_tecnico"
    elif 'superior' in x['nivel_edu'].tolist() and 'bachiller' in x['diploma'].tolist():
        return "superior_bachiller"
    elif 'superior' in x['nivel_edu'].tolist() and 'ninguno' in x['diploma'].tolist():
        return "superior"
    elif 'superior' in x['nivel_edu'].tolist() and 'no_informa' in x['diploma'].tolist():
        return "superior"
    elif 'media' in x['nivel_edu'].tolist() and 'bachiller' in x['diploma'].tolist():
        return "media_bachiller"
    elif 'media' in x['nivel_edu'].tolist() and 'ninguno' in x['diploma'].tolist():
        return "media"
    elif 'media' in x['nivel_edu'].tolist() and 'no_informa' in x['diploma'].tolist():
        return "media"
    else:
        return x['nivel_edu'][0]



In [4]:
df_familia=df.groupby('llave').apply(lambda x : pd.Series({
    'cant_indiv':x['edad'].count(),
    'coef':ss.variation(x["edad"]),
    'ciudad_origen':x['ciudad_origen'][0],
    'tipo':tipo(x),
    'edad_mediana':np.floor(x['edad'].median()),
    'rango_edad':rango_edad(x),
    'estrato':x['estrato'][0],
    'ingreso_hogar': x['ingreso_hogar'].mean(),
    'rango_ingreso_hog': x['rango_ingreso_hog'][0],
    'estado_civ': estado_civil(x),
    'nivel_edu':educacion(x),
    'viajo': x['viajo'][0],
    'interno':x['interno'][0],    
#     
    'cuantos_int':  x['cuantos_int'][0],
    'depto_dest': x['depto_dest'][0],
    'mpio_dest': x['mpio_dest'][0],
    'fecha_ini': x['fecha_ini'][0],
    'fecha_fin': x['fecha_fin'][0],
    'pernoct': x['pernoct'][0],
    'motivo_int': x['motivo_int'][0],   
    'alojamiento': alojamiento(x),
    'med_transp': x['med_transp'][0],
    'gasto_tot': x['gasto_tot'][0],
    'motivo_no':x['motivo_no'][0],
    'gast_aloj':x['gast_aloj'][0],
    'valgast_aloj':x['valgast_aloj'][0],
    'gast_public':x['gast_public'][0],
    'valgast_public':x['valgast_public'][0],
    'gast_alim':x['gast_alim'][0],
    'gast_aereo':x['gast_aereo'][0],
    'valgast_aereo':x['valgast_aereo'][0],
    'excursion':x['excursion'][0],
    'excur_extranj':x['excur_extranj'][0],
    'excur_int':x['excur_int'][0],
    'excur_motiv':x['excur_motiv'][0],
    'excur_deptodest':x['excur_deptodest'][0],
    'excur_mpiodest':x['excur_mpiodest'][0],
    'excur_medtransp':x['excur_medtransp'][0],
    'excur_valgasttransp':x['excur_valgasttransp'][0],
    'excur_valgastpublic':x['excur_valgastpublic'][0],
    'excur_valgastalim':x['excur_valgastalim'][0]    
}))

In [5]:
df_familia

,cant_indiv,coef,ciudad_origen,tipo,edad_mediana,rango_edad,estrato,ingreso_hogar,rango_ingreso_hog,estado_civ,nivel_edu,viajo,interno,cuantos_int,depto_dest,mpio_dest,fecha_ini,fecha_fin,pernoct,motivo_int,alojamiento,med_transp,gasto_tot,motivo_no,gast_aloj,valgast_aloj,gast_public,valgast_public,gast_alim,gast_aereo,valgast_aereo,excursion,excur_extranj,excur_int,excur_motiv,excur_deptodest,excur_mpiodest,excur_medtransp,excur_valgasttransp,excur_valgastpublic,excur_valgastalim
llave,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3000000_1,3,0.823690,Medellín,familia,25.0,25_30,1,880000.0,1_2,familia,media_bachiller,no,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000001_1,3,0.635001,Medellín,familia,14.0,10_15,1,860000.0,1_2,familia,secundaria,no,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000002_1,1,0.000000,Medellín,mujer,68.0,65_70,1,1050000.0,1_2,viudo,primaria,no,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,recursos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000003_1,3,0.593015,Medellín,familia,21.0,20_25,1,820000.0,0_1,familia,media_bachiller,no,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,recursos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000004_1,4,0.435135,Medellín,familia,19.0,15_20,1,850000.0,1_2,familia,media_bachiller,no,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,recursos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033697_1,1,0.000000,Armenia,mujer,63.0,60_65,1,150000.0,0_1,soltero,secundaria,no,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,recursos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3033698_1,3,0.334398,Cali,familia,37.0,35_40,3,1500000.0,1_2,familia,superior_univerisitario,no,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,tiempo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3034791_1,1,0.000000,Pereira,hombre,27.0,25_30,4,1700000.0,2_3,soltero,superior_univerisitario,si,si,1,Bolívar,13001.0,10/06/2019 0:00,15/06/2019 0:00,5.0,negocios,None,particular,800000.0,NaN,no_responde,NaN,no_responde,NaN,no_responde,no_responde,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_familia[df_familia['viajo']=='si']

,cant_indiv,coef,ciudad_origen,tipo,edad_mediana,rango_edad,estrato,ingreso_hogar,rango_ingreso_hog,estado_civ,nivel_edu,viajo,interno,cuantos_int,depto_dest,mpio_dest,fecha_ini,fecha_fin,pernoct,motivo_int,alojamiento,med_transp,gasto_tot,motivo_no,gast_aloj,valgast_aloj,gast_public,valgast_public,gast_alim,gast_aereo,valgast_aereo,excursion,excur_extranj,excur_int,excur_motiv,excur_deptodest,excur_mpiodest,excur_medtransp,excur_valgasttransp,excur_valgastpublic,excur_valgastalim
llave,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3000007_1,1,0.000000,Medellín,mujer,55.0,55_60,1,557000.0,0_1,viudo,ninguno,si,si,1,Antioquia,5353.0,15/01/2019 0:00,22/01/2019 0:00,7.0,visita,None,publico,50000.0,NaN,no,NaN,no,NaN,no,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000019_1,2,0.049180,Medellín,familia,61.0,60_65,3,0.0,no_reporta,familia,superior_univerisitario,si,si,1,Magdalena,47001.0,10/01/2019 0:00,14/01/2019 0:00,4.0,vacaciones,None,aereo,4000000.0,NaN,si,1000000.0,si,100000.0,si,si,400000.0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000021_1,1,0.000000,Medellín,mujer,56.0,55_60,3,1000000.0,1_2,separado,primaria,si,si,1,Caldas,17380.0,22/12/2018 0:00,8/01/2019 0:00,17.0,vacaciones,None,publico,1000000.0,NaN,no,NaN,si,50000.0,si,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000022_1,3,0.620093,Medellín,familia,30.0,30_35,3,2600000.0,3_5,familia,superior_tecnico,si,si,1,Antioquia,5887.0,30/12/2018 0:00,2/01/2019 0:00,3.0,visita,None,particular,250000.0,NaN,si,35000.0,no,NaN,si,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3000024_1,2,0.447368,Medellín,familia,38.0,35_40,3,1000000.0,1_2,familia,superior_bachiller,si,si,1,Antioquia,5282.0,1/01/2019 0:00,7/01/2019 0:00,6.0,visita,None,publico,300000.0,NaN,no,NaN,no,NaN,si,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3033687_1,2,0.292683,Cúcuta,familia,41.0,40_45,3,820000.0,0_1,familia,media_bachiller,si,si,1,Norte de Santander,54172.0,29/06/2019 0:00,30/06/2019 0:00,1.0,vacaciones,None,publico,100000.0,NaN,no,NaN,no,NaN,si,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3033692_1,4,0.514224,Ibagué,familia,38.0,35_40,3,1500000.0,1_2,familia,superior_tecnico,si,si,1,Tolima,73067.0,10/06/2019 0:00,27/06/2019 0:00,17.0,visita,None,publico,178000.0,NaN,no,NaN,si,24000.0,si,no,NaN,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3033696_1,7,0.644191,Quibdó,familia,23.0,20_25,3,1100000.0,1_2,familia,superior_univerisitario,si,si,1,"Bogotá, D.C.",11001.0,10/06/2019 0:00,29/06/2019 0:00,19.0,salud,None,aereo,600000.0,NaN,no,NaN,si,100000.0,si,si,399000.0,no,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df_familia.to_csv('data_familias.csv')

In [31]:
x=df_familia['ciudad_origen'].unique().tolist()
df22 = pd.DataFrame(np.array(x), columns = ["ciudad"])
df22

,ciudad
0,Medellín
1,Barranquilla
2,Bogotá
3,Cartagena
4,Tunja
5,Manizales
6,Florencia
7,Popayán
8,Valledupar
9,Monteria


In [59]:
import geocoder
a='San Andrés Colombia'
g = geocoder.osm(a)
print(g.osm['x'],g.osm['y'])

-75.2000115 5.5966647
